# Minimal TSMNet demo notebook for inter-session/-subject source-free (SF) offline and online unsupervised domain adaptation (UDA)

In [1]:
import torch
import sklearn
import pandas as pd


from moabb.datasets import Kalunga2016, MAMEM1, MAMEM2, MAMEM3, Lee2019_SSVEP
from moabb.paradigms import SSVEP


# from library.utils.torch import StratifiedDomainDataLoader
from spdnets.utils.data import StratifiedDomainDataLoader, DomainDataset

from spdnets.trainer import Trainer
from spdnets.callbacks import EarlyStopping, MomentumBatchNormScheduler
from spdnets.model import GyroNet
import os
import numpy as np
from scipy.linalg import sqrtm, inv 
import spdnets.functionals as fn
from copy import deepcopy
import spdnets.batchnorm as bn

In [2]:
def euler_align(raw_array):
    """
    Perform Euler alignment on input numpy array ([trials * channels * samples]) 
    and return the aligned array.
    """
    # Calculate mean covariance matrix
    cov_matrices = [np.cov(trial, rowvar=True) for trial in raw_array]
    #cov_matrices = np.cov(raw_array,axis=0, rowvar=True)
    mean_cov_matrix = np.mean(cov_matrices, axis=0)
    
    # Compute transformation matrix
    trans_matrix = inv(sqrtm(mean_cov_matrix))
    
    # Apply transformation to all trials using broadcasting
    return trans_matrix @ raw_array

In [3]:
# network and training configuration
cfg = dict(
    epochs = 100,
    batch_size_train = 50,
    domains_per_batch = 5,
    validation_size = 0.2,
    evaluation = 'inter-subject', # or 'inter-subject'
    dtype = torch.float64,
    training=True, 
    swd_loss_weight=0,
    euler_align=False,
    lr=0.001,
    weight_decay=1e-4,
    mdl_kwargs = dict( 
    bnorm_dispersion=bn.BatchNormDispersion.SCALAR,
    domain_adaptation=True
)
)

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('GPU')
else:
    device = torch.device('cpu')
    print('CPU')

GPU


In [4]:
def split_eeg_data(X, labels, metadata, original_duration=2, target_duration=1):
    n_trials, n_channels, n_samples = X.shape
    
    # 计算采样率（假设数据是均匀采样的）
    sampling_rate = n_samples / original_duration
    target_samples = int(target_duration * sampling_rate)
    
    # 计算每个trial可以拆分成多少个片段
    n_segments = int(original_duration / target_duration)
    
    # 初始化拆分后的数据
    X_split = []
    labels_split = []
    metadata_split = []
    
    for i in range(n_trials):
        for j in range(n_segments):
            start_idx = j * target_samples
            end_idx = start_idx + target_samples
            
            # 提取数据片段
            segment = X[i, :, start_idx:end_idx]
            X_split.append(segment)
            
            # 复制标签（同一个trial的所有片段标签相同）
            labels_split.append(labels[i])
            
            # 复制并修改元数据
            meta_row = metadata.iloc[i].copy()
            meta_row['original_trial_idx'] = i
            meta_row['segment_idx'] = j
            meta_row['trial_segment'] = f"{i}_{j}"
            metadata_split.append(meta_row)
    
    X_split = np.array(X_split)
    labels_split = np.array(labels_split)
    metadata_split = pd.DataFrame(metadata_split).reset_index(drop=True)
    
    return X_split, labels_split, metadata_split

In [5]:
def sfuda_offline(dataset : DomainDataset, model : GyroNet):
    model.eval()
    model.domainadapt_finetune(dataset.features.to(dtype=cfg['dtype'], device=device), dataset.labels.to(device=device), dataset.domains, None)

## load a MOABB dataset

In [6]:
# moabb_ds = Kalunga2016()
# n_classes = 4
# dataset='Kalunga2016'
# original_duration =2
# target_duration = 1
# moabb_paradigm = SSVEP(n_classes=n_classes, fmin=1, fmax=50, resample=None)


moabb_ds = Lee2019_SSVEP()
n_classes = 4
dataset='Lee2019_SSVEP'
original_duration =4
target_duration = 1
moabb_paradigm = SSVEP(n_classes=n_classes, fmin=1, fmax=50, resample=256)

# moabb_ds = MAMEM1()
# n_classes = 5
# dataset='MAMEM1'
# original_duration =3
# target_duration = 1
# moabb_paradigm = SSVEP(n_classes=n_classes, fmin=1, fmax=50, resample=256)
# sampling_rate = 256

# moabb_ds = MAMEM2()
# n_classes = 5
# dataset='MAMEM2'
# original_duration =3
# target_duration = 1
# moabb_paradigm = SSVEP(n_classes=n_classes, fmin=1, fmax=50, resample=256)
# sampling_rate = 256

# moabb_ds = MAMEM3()
# n_classes = 5
# dataset='MAMEM3'
# original_duration =3
# target_duration = 1
# moabb_paradigm = SSVEP(n_classes=n_classes, fmin=1, fmax=50, resample=None)
# sampling_rate = 128

Choosing the first 4 classes from all possible events


## fit and evaluat the model for all domains

In [ ]:
random_seed=42
torch.manual_seed(random_seed)
model_name = 'GyroNet'
for swd_weight in [0,0.001,0.005,0.01,0.05,0.1,0.2]:
    for ea_align in [0]:
        for lr in [0.003, 0.005]:
            records = []
            records1=[]
            if 'inter-session' in cfg['evaluation']:
                subset_iter = iter([[s] for s in moabb_ds.subject_list])
                groupvarname = 'session'
                eval_fashion = 'session'
            elif 'inter-subject' in cfg['evaluation']:
                subset_iter = iter([None])
                groupvarname = 'subject'
                eval_fashion = 'subject'
            else:
                raise NotImplementedError()
            location = f'swd_weight_{swd_weight}_EA_{ea_align}_lr_{lr}'
            index = 0
            while os.path.exists(f'pretrained/{dataset}/{location}/{index}'):
                index += 1
            # iterate over subject groups
            global_manifold_violation_counters = {}
            global_total_forward_calls = 0
            
            for ix_subset, subjects in enumerate(subset_iter):

                # get the data from the MOABB paradigm/dataset
                X, labels, metadata = moabb_paradigm.get_data(moabb_ds, subjects=subjects, return_epochs=False)
                X, labels, metadata = split_eeg_data(X, labels, metadata)


                # extract domains = subject/session
                metadata['label'] = labels
                metadata['domain'] = metadata.apply(lambda row: f'{row.subject}/{row.session}',  axis=1)
                domain = sklearn.preprocessing.LabelEncoder().fit_transform(metadata['domain'])

                # convert to torch tensors
                domain = torch.from_numpy(domain)
                if ea_align ==1:
                    for i in domain.unique():
                        X[domain == i] = euler_align(X[domain == i])
                    input_align = 'ea'
                else:
                    input_align = 'none'
                # for du in domain.unique():
                #     domain_ixs = domain == du
                #     X[domain_ixs] = fn.robust_zscore(X[domain_ixs], per_channel_variance=False, axis=-1)
                # X = np.clip(X, -5, 5)
                if lr >= 0.01:
                    weight_decay = 1e-3
                else:
                    weight_decay = 1e-4  


                X = torch.from_numpy(X)
                y = sklearn.preprocessing.LabelEncoder().fit_transform(labels)
                y = torch.from_numpy(y)

                lab,lab_count = y.unique(return_counts=True)    

                # leave one subject or session out
                if 'inter-session' in cfg['evaluation']:
                    domain_count = len(domain.unique())
                elif 'inter-subject' in cfg['evaluation']:
                    domain_count = metadata['subject'].nunique()
                if domain_count <10:
                    cv_outer = sklearn.model_selection.LeaveOneGroupOut()
                else:
                    cv_outer = sklearn.model_selection.GroupKFold(n_splits=10)
                cv_outer_group = metadata[groupvarname]

                # train/validation split stratified across domains and labels
                cv_inner_group = metadata.apply(lambda row: f'{row.domain}/{row.label}',  axis=1)
                cv_inner_group = sklearn.preprocessing.LabelEncoder().fit_transform(cv_inner_group)

                # add datadependen model kwargs
                mdl_kwargs = deepcopy(cfg['mdl_kwargs'])
                mdl_kwargs['num_classes'] = n_classes
                mdl_kwargs['num_electrodes'] = X.shape[1]
                mdl_kwargs['chunk_size'] = X.shape[2]
                mdl_kwargs['domains'] = domain.unique()
                # perform outer CV
                for ix_fold, (fit, test) in enumerate(cv_outer.split(X, y, cv_outer_group)):
                    # if ix_fold not in [0,1,2,3,4]:
                    #     continue

                    # split fitting data into train and validation 
                    cv_inner = sklearn.model_selection.StratifiedShuffleSplit(n_splits=1, test_size=cfg['validation_size'])
                    train, val = next(cv_inner.split(X[fit], y[fit], cv_inner_group[fit]))

                    # adjust number of 
                    du = domain[fit][train].unique()
                    if cfg['domains_per_batch'] > len(du):
                        domains_per_batch = len(du)
                    else:
                        domains_per_batch = cfg['domains_per_batch']

                    # split entire dataset into train/validation/test
                    ds_train = DomainDataset(X[fit][train], y[fit][train], domain[fit][train])
                    ds_val = DomainDataset(X[fit][val], y[fit][val], domain[fit][val])
                    

                    # create dataloaders
                    # for training use specific loader/sampler so taht 
                    # batches contain a specific number of domains with equal observations per domain
                    # and stratified labels
                    loader_train = StratifiedDomainDataLoader(ds_train, cfg['batch_size_train'], domains_per_batch=domains_per_batch, shuffle=True, drop_last = False)
                    #loader_train= torch.torch.utils.data.DataLoader(ds_train,50)
                    loader_val = torch.utils.data.DataLoader(ds_val, batch_size=len(ds_val))
                    test_domain = metadata['domain'].iloc[test].unique()

                    # create the model

                    model = GyroNet(device=device,dtype=cfg['dtype'],**mdl_kwargs).to(device=device, dtype=cfg['dtype'])
                    es = EarlyStopping(metric='val_loss', higher_is_better=False, patience=20, verbose=False)
                    
                    bn_sched = MomentumBatchNormScheduler(
                        epochs=cfg['epochs']-10,
                        bs0=cfg['batch_size_train'],
                        bs=cfg['batch_size_train']/cfg['domains_per_batch'], 
                        tau0=0.85
                    )

                        # create the trainer
                    trainer = Trainer(
                        max_epochs= cfg['epochs'],
                        min_epochs= 70,
                        callbacks=[es,bn_sched],
                        loss=torch.nn.CrossEntropyLoss(),
                        device=device, 
                        dtype=torch.float64,
                        swd_weight=swd_weight,
                        lr=lr,
                        weight_decay=weight_decay
                    )
                    # fit the modelzz

                    # print parameters
                    if cfg['training']:
                        save_dir = f'pretrained/{dataset}/{location}/{index}'
                        trainer.fit(model, train_dataloader=loader_train, val_dataloader=loader_val)
                        model.print_manifold_violation_stats()
                        os.makedirs(save_dir, exist_ok=True)
                        torch.save(model, f'pretrained/{dataset}/{location}/{index}/{ix_subset}_{ix_fold}.pt')
                        print(f'ES best epoch={es.best_epoch}')
                        res = trainer.test(model, dataloader=loader_train)
                        records1.append(dict(input=input_align,latent='gyro',lr=lr,swd_weight=swd_weight,model=model_name,mode='train',dataset=dataset,**res))
                        res = trainer.test(model, dataloader=loader_val)
                        records1.append(dict(input=input_align,latent='gyro',lr=lr,swd_weight=swd_weight,model=model_name,mode='validation',dataset=dataset,**res))
                    else:
                        pass
                    sfuda_offline_net=torch.load(f'pretrained/{dataset}/{location}/{index}/{ix_subset}_{ix_fold}.pt', map_location=device, weights_only=False)


                    global_total_forward_calls += model.total_forward_calls
                    for layer_name, violation_count in model.manifold_violation_counters.items():
                        if layer_name not in global_manifold_violation_counters:
                            global_manifold_violation_counters[layer_name] = 0
                        global_manifold_violation_counters[layer_name] += violation_count
                    # evaluation

                    test_domain=domain[test].unique()
                    for test_domain in test_domain:
                        if 'inter-session' in cfg['evaluation']:
                            subject=ix_subset
                        else:
                            subject=ix_fold
                        print(f"Subject:{subject}, test domain: {test_domain}")    
                        ds_test = DomainDataset(X[test][domain[test] == test_domain], y[test][domain[test] == test_domain], domain[test][domain[test] == test_domain])
                        loader_test = torch.utils.data.DataLoader(ds_test, batch_size=len(ds_test))
                        sfuda_offline(ds_test, sfuda_offline_net)
                        res = trainer.test(sfuda_offline_net, dataloader=loader_test)
                        res2 = res
                        print(f"{model_name},Test results: {res2}")
                        records.append(dict(input=input_align,latent='gyro',lr=lr,swd_weight=swd_weight,model=model_name,dataset=dataset,subject=subject,domain=test_domain, **res))

            # save records
            resdf = pd.DataFrame(records)
            resdf.to_csv(f'pretrained/{dataset}/results_{location}_{index}.csv',index=False)
            resdf1 = pd.DataFrame(records1)
            resdf1.to_csv(f'pretrained/{dataset}/train_val_results_{location}_{index}.csv',index=False)


/home/dbi/shanglin/miniforge3/envs/gyronet/lib/python3.9/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 100 events (all good), 0 – 4 s (baseline off), ~189.3 MB, data loaded,
 '12.0': 25
 '8.57': 25
 '6.67': 25
 '5.45': 25>
  warn(f"warnEpochs {epochs}")
/home/dbi/shanglin/miniforge3/envs/gyronet/lib/python3.9/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 100 events (all good), 0 – 4 s (baseline off), ~189.3 MB, data loaded,
 '12.0': 25
 '8.57': 25
 '6.67': 25
 '5.45': 25>
  warn(f"warnEpochs {epochs}")
/home/dbi/shanglin/miniforge3/envs/gyronet/lib/python3.9/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 100 events (all good), 0 – 4 s (baseline off), ~189.3 MB, data loaded,
 '12.0': 25
 '8.57': 25
 '6.67': 25
 '5.45': 25>
  warn(f"warnEpochs {epochs}")
/home/dbi/shanglin/miniforge3/envs/gyronet/lib/python3.9/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: 

epoch=  0 gd-step=  297 trn_loss= 0.4271 trn_score=0.8611 val_loss= 0.4475 val_score=0.8516 
epoch= 10 gd-step= 3262 trn_loss= 0.1543 trn_score=0.9476 val_loss= 0.1557 val_score=0.9466 
epoch= 20 gd-step= 6228 trn_loss= 0.1394 trn_score=0.9532 val_loss= 0.1632 val_score=0.9437 
epoch= 30 gd-step= 9189 trn_loss= 0.1269 trn_score=0.9577 val_loss= 0.1363 val_score=0.9521 
epoch= 40 gd-step=12150 trn_loss= 0.1374 trn_score=0.9554 val_loss= 0.1528 val_score=0.9513 
epoch= 50 gd-step=15116 trn_loss= 0.1459 trn_score=0.9483 val_loss= 0.1582 val_score=0.9466 
epoch= 60 gd-step=18082 trn_loss= 0.1576 trn_score=0.9482 val_loss= 0.1735 val_score=0.9419 
epoch= 70 gd-step=21046 trn_loss= 0.1164 trn_score=0.9581 val_loss= 0.1460 val_score=0.9497 


/home/to-server/shanglin/projects/HEEG/SSVEP/Gyro/man_0/spdnets/callbacks.py:121: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(path))



MANIFOLD VIOLATION STATISTICS
Total forward calls: 42751
------------------------------------------------------------
after_projx         :      0 violations (  0.00%)
after_lc1           :      0 violations (  0.00%)
after_bn            :      0 violations (  0.00%)
after_elu           :      0 violations (  0.00%)
after_avpool        :      0 violations (  0.00%)
after_flatten       :      0 violations (  0.00%)
ES best epoch=30
Subject:0, test domain: 10
GyroNet,Test results: {'loss': 0.0421762901322648, 'score': 0.99}
Subject:0, test domain: 11
GyroNet,Test results: {'loss': 0.7597191534031924, 'score': 0.6799999999999999}
Subject:0, test domain: 32
GyroNet,Test results: {'loss': 0.12137799397893806, 'score': 0.95}
Subject:0, test domain: 33
GyroNet,Test results: {'loss': 0.24450398151735078, 'score': 0.94}
Subject:0, test domain: 54
GyroNet,Test results: {'loss': 0.6251006890310008, 'score': 0.775}
Subject:0, test domain: 55
GyroNet,Test results: {'loss': 0.2515561473961161, 'sco

/home/dbi/shanglin/miniforge3/envs/gyronet/lib/python3.9/site-packages/sklearn/model_selection/_split.py:2335: UserWarning: The groups parameter is ignored by StratifiedShuffleSplit
  warnings.warn(


...

epoch=  0 gd-step=  298 trn_loss= 0.3935 trn_score=0.8641 val_loss= 0.4083 val_score=0.8557 
epoch= 10 gd-step= 3278 trn_loss= 0.2350 trn_score=0.9313 val_loss= 0.2711 val_score=0.9174 
epoch= 20 gd-step= 6257 trn_loss= 0.2236 trn_score=0.9242 val_loss= 0.2450 val_score=0.9164 
epoch= 30 gd-step= 9236 trn_loss= 0.3003 trn_score=0.9050 val_loss= 0.3369 val_score=0.8990 
epoch= 40 gd-step=12212 trn_loss= 0.1732 trn_score=0.9476 val_loss= 0.2024 val_score=0.9414 
epoch= 50 gd-step=15190 trn_loss= 0.1457 trn_score=0.9509 val_loss= 0.1746 val_score=0.9409 
epoch= 60 gd-step=18169 trn_loss= 0.1345 trn_score=0.9541 val_loss= 0.1660 val_score=0.9458 
epoch= 70 gd-step=21147 trn_loss= 0.1238 trn_score=0.9577 val_loss= 0.1662 val_score=0.9451 
epoch= 80 gd-step=24128 trn_loss= 0.1338 trn_score=0.9529 val_loss= 0.1636 val_score=0.9411 


/home/to-server/shanglin/projects/HEEG/SSVEP/Gyro/man_0/spdnets/callbacks.py:121: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(path))



MANIFOLD VIOLATION STATISTICS
Total forward calls: 51322
------------------------------------------------------------
after_projx         :      0 violations (  0.00%)
after_lc1           :      0 violations (  0.00%)
after_bn            :      0 violations (  0.00%)
after_elu           :      0 violations (  0.00%)
after_avpool        :      0 violations (  0.00%)
after_flatten       :      0 violations (  0.00%)
ES best epoch=65
Subject:0, test domain: 10
GyroNet,Test results: {'loss': 0.026325003755534905, 'score': 1.0}
Subject:0, test domain: 11
GyroNet,Test results: {'loss': 0.6968876893408061, 'score': 0.7650000000000001}
Subject:0, test domain: 32
GyroNet,Test results: {'loss': 0.0643275935698463, 'score': 0.985}
Subject:0, test domain: 33
GyroNet,Test results: {'loss': 0.1994054860192596, 'score': 0.98}
Subject:0, test domain: 54
GyroNet,Test results: {'loss': 0.5914235491752503, 'score': 0.8250000000000001}
Subject:0, test domain: 55
GyroNet,Test results: {'loss': 0.268644622

/home/dbi/shanglin/miniforge3/envs/gyronet/lib/python3.9/site-packages/sklearn/model_selection/_split.py:2335: UserWarning: The groups parameter is ignored by StratifiedShuffleSplit
  warnings.warn(


epoch=  0 gd-step=  296 trn_loss= 0.2922 trn_score=0.9074 val_loss= 0.3021 val_score=0.9026 
epoch= 10 gd-step= 3266 trn_loss= 0.1479 trn_score=0.9502 val_loss= 0.1684 val_score=0.9443 
